In [5]:
import os
import warnings
import numpy as np
from scipy.io import wavfile
from hmmlearn import hmm
from features import mfcc

In [6]:
class ModelHMM(object) : 
    def __init__(self, num_components=4, num_iter=1000) : 
        self.n_components = num_components
        self.n_iter = num_iter
        
        self.model_name = "GaussianHMM" #HMM의 유형
        self.cov_type = 'diag' #HMM의 공분산 유형
        self.models = []
        
        self.model = hmm.GaussianHMM(n_components = self.n_components, covariance_type=self.cov_type, n_iter=self.n_iter)
        
    
    def train(self, training_data) : 
        np.seterr(all="ignore")
        cur_model = self.model.fit(training_data)
        self.models.append(cur_model)

In [7]:
def build_models(input_folder) : 
    speech_models = []

    for dirname in os.listdir(input_folder) : 
        subfolder = os.path.join(input_folder, dirname)

        if not os.path.isdir(subfolder) :
            continue

        label = subfolder[subfolder.rfind("/") + 1:]

        X = np.array([]) #variable initialization
        training_files = [x for x in os.listdir(subfolder) if x.endswith(".wav")][:-1]

        for filename in training_files : 
            filepath = os.path.join(subfolder, filename) 
            sampling_freq, signam = wavfile.read(filepath)

            with warnings.catch_warnings() : 
                warnings.simplefilter("ignore") 
                features_mfcc = mfcc(signal, sampling_freq)

            if len(X) == 0 :
                X = features_mfcc
            else : 
                X = np.append(X, features_mfcc, axis=0)

        model = ModelHMM() #creating HMM model
        with arnings.catch_warnings() : #HMM model learning
            warnings.simplefilter("ignore")
            model.train(X)
            
        speech_models.append((model, label))
        model = None
            
    return speech_models

In [8]:
def run_test(test_files) : 
    for test_file in test_files : #divide input data
        sampling_freq, signal = wavfile.read(test_file) #load input data
        
        #extract MFCC features
        with warnings.catch_warnings() : 
            warnings.simplefilter("ignore")
            features_mfcc = mfcc(signal, sampling_freq)
            
        max_score = -float("inf")
        output_label = None
        
        for item in speech_models : 
            model, label = item
            
            score = model.compute_score(features_mfcc) 
            if score > max_score : 
                max_score = score
                predicted_label = label
                
        start_index = test_file.find("/") + 1
        end_index = test_file.rfind("/")
        original_label = test_file[start_index : end_index]
        print("\nOriginal:", original_label)
        print("\Predicted:", predicted_label)

In [9]:
input_folder = data

NameError: name 'data' is not defined

In [ ]:
speech_models = build_models(input_folder) 
test_files = []
for root, dirs, files in os.walk(input_folder) : 
    for filename in (x for x in files if "15" in x) : 
        filepath = os.path.join(root, filename)
        test_files.append(filepath)

In [ ]:
run_tests(test_files)